In [55]:
!pip install pdfplumber
!pip install pymupdf

In [56]:
from google.colab import files
import os

# Upload the zip file
uploaded = files.upload()

Saving 2033-AM.pdf to 2033-AM (1).pdf


In [57]:
from collections import namedtuple
import pandas as pd
import pdfplumber
import re

In [58]:
import fitz
import os

In [59]:
for filename in uploaded.keys():
    output_folder = "output_pages"
    os.makedirs(output_folder, exist_ok=True)

    doc = fitz.open(filename)
    count = 0
    for i, page in enumerate(doc):
        text = page.get_text("text")
        lines = text.splitlines()
        if lines and "HCM Signalized Intersection Capacity Analysis" in lines[0]:
            new_doc = fitz.open()
            new_doc.insert_pdf(doc, from_page=i, to_page=i)
            output_path = os.path.join(output_folder, f"HCM_Signalized_{count+1}.pdf")
            new_doc.save(output_path)
            new_doc.close()
            count += 1

In [77]:
Output = pd.DataFrame(columns= ['Scenario','Node','V/C', 'Delay', 'LOS', 'W_Movement','W_Delay','W_V/C','W_LOS'] )
for filename in os.listdir(output_folder):
    file_path = os.path.join(output_folder, filename)
    all_text = []
    with pdfplumber.open(file_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            text = page.extract_text( layout=True)
            if text:  # check if text is not None
                all_text.append(text)

    # Join all page texts into one string
    full_text = '\n'.join(all_text)

    data = []
    for line in full_text.split('\n'):
        data.append(line)

    node= data[4][9:30]
    Scenario= data[4][32:]
    stripped_lines = [line.strip() for line in data[45:47]]
    control_delay= stripped_lines[0].split(" ")[5]
    level_of_service = stripped_lines[0].split(" ")[-1]
    v_c_ratio = stripped_lines[1].split(" ")[-1]

    lines = [data[i].strip(" ")[17:] for i in  [8,35, 39, 40]]
    header_line = lines[0]
    col_starts = [i for i in range(len(header_line)) if (i == 0 or header_line[i-1] == ' ') and header_line[i] != ' ']
    col_ranges = [(col_starts[i], col_starts[i+1] if i+1 < len(col_starts) else None) for i in range(len(col_starts))]
    headers = [header_line[start:end].strip() for start, end in col_ranges]
    table = []
    for line in lines[1:]:
        row = [line[start:end].strip() if line[start:end].strip() else 'N/A' for start, end in col_ranges]
        table.append(row)
    df = pd.DataFrame(table, columns=headers)
    delay_row = pd.to_numeric(df.iloc[1], errors='coerce')
    print(delay_row)

EBL    20.5
EBR     NaN
NBU     NaN
NBL    19.1
NBT    19.2
SBU     NaN
SBT    20.6
SBR     NaN
Name: 1, dtype: float64
WBL    25.8
WBR     1.6
NBT     NaN
NBR     0.3
SBL    26.2
SBT     NaN
Name: 1, dtype: float64
EBL     NaN
EBT    25.0
EBR     0.7
WBL     NaN
WBT    30.8
WBR     2.0
NBL     5.0
NBT     NaN
NBR     NaN
SBL     NaN
SBT     NaN
SBR     NaN
Name: 1, dtype: float64
EBT    24.2
EBR     0.2
WBU     NaN
WBL    21.3
WBT    26.2
NBL    30.0
NBR     NaN
Name: 1, dtype: float64
EBT    21.9
EBR     0.1
WBL     NaN
WBT     9.3
NBL    24.3
NBR     NaN
Name: 1, dtype: float64
EBT    34.8
EBR     NaN
WBL     NaN
WBT    31.2
NBL     NaN
NBR     NaN
Name: 1, dtype: float64
EBL     NaN
EBT    30.4
EBR     NaN
WBL     NaN
WBT    30.1
WBR     2.0
NBL     8.2
NBT     NaN
NBR     NaN
SBL     NaN
SBT    30.4
SBR     NaN
Name: 1, dtype: float64
WBL    22.3
WBR    22.1
NBT    24.7
NBR     NaN
SBL     NaN
SBT    24.4
Name: 1, dtype: float64
EBL     NaN
EBT    39.6
EBR     NaN
WBL     NaN
WBT 

In [71]:
Output = pd.DataFrame(columns= ['Scenario','Node','V/C', 'Delay', 'LOS', 'W_Movement','W_Delay','W_V/C','W_LOS'] )
for filename in os.listdir(output_folder):
    file_path = os.path.join(output_folder, filename)
    all_text = []
    with pdfplumber.open(file_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            text = page.extract_text( layout=True)
            if text:  # check if text is not None
                all_text.append(text)

    # Join all page texts into one string
    full_text = '\n'.join(all_text)

    data = []
    for line in full_text.split('\n'):
        data.append(line)

    node= data[4][9:30]
    Scenario= data[4][32:]
    stripped_lines = [line.strip() for line in data[45:47]]
    control_delay= stripped_lines[0].split(" ")[5]
    level_of_service = stripped_lines[0].split(" ")[-1]
    v_c_ratio = stripped_lines[1].split(" ")[-1]

    lines = [data[i].strip(" ")[17:] for i in  [8,35, 39, 40]]
    header_line = lines[0]
    col_starts = [i for i in range(len(header_line)) if (i == 0 or header_line[i-1] == ' ') and header_line[i] != ' ']
    col_ranges = [(col_starts[i], col_starts[i+1] if i+1 < len(col_starts) else None) for i in range(len(col_starts))]
    headers = [header_line[start:end].strip() for start, end in col_ranges]
    table = []
    for line in lines[1:]:
        row = [line[start:end].strip() if line[start:end].strip() else 'N/A' for start, end in col_ranges]
        table.append(row)
    df = pd.DataFrame(table, columns=headers)
    delay_row = pd.to_numeric(df.iloc[1], errors='coerce')

    max_col = delay_row.idxmax()
    vc = df.loc[0, max_col]
    delay = df.loc[1, max_col]
    los = df.loc[2, max_col]

    new_row = {
        'Scenario': Scenario,
        'Node': node,
        'V/C':v_c_ratio,
        'Delay': control_delay,
        'LOS': level_of_service,
        'W_Movement': max_col,
        'W_Delay': delay,
        'W_V/C': vc,
        'W_LOS': los
    }

    Output = pd.concat([Output, pd.DataFrame([new_row])], ignore_index=True)

<ipython-input-71-102fd580492c>:37: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  max_col = delay_row.idxmax()


KeyError: nan

In [33]:
Output

,Scenario,Node,V/C,Delay,LOS,W_Movement,W_Delay,W_V/C,W_LOS
0,...,JN 1:,1.03,43.6,D,EBL,83.2,1.04,F
1,...,2:,1.05,42.1,D,SWL,69.0,0.99,E
2,...,3:,0.61,8.8,A,SEL,18.9,0.45,B
3,...,4:,0.72,22.0,C,SEL,33.1,0.44,C
4,...,5:,1.24,14.2,B,NWT,25.8,0.19,C
5,...,6:,0.70,14.1,B,NWL,26.2,0.77,C
6,...,7:,0.60,26.2,C,NWL,42.9,0.98,D
7,...,8:,0.84,30.3,C,SWL,63.9,0.36,E
8,...,9:,1.79,411.9,F,NWT,533.9,2.07,F
9,...,10:,1.53,252.0,F,NER,415.9,1.77,F


In [34]:
Output.to_excel('Synchro_Report.xlsx', index=False)